<a href="https://colab.research.google.com/github/weprintmoney/LLMPractice/blob/main/9.02%20Canadian%20Law%20LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Custom RAG Implementation</center></h1>
<h2><center>The Consolidated Acts and Regulations of Canada</center></h2>
<h3><center>Charlcye Mitchell & Matt Moore, May 2024</center></h3>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb)

### Objective
The goal of this assignment is to explore advanced applications of large language models in the legal domain. We will implement the LLaMA3 GGUF model with the Retrieval-Augmented Generation (RAG) technique, using a dataset consisting of the consolidated acts and regulations of Canada. This implementation aims to leverage the rich contextual understanding of the LLaMA3 model with the retrieval capabilities of RAG to enhance the accuracy and relevance of generated responses in legal contexts.

### Background

*   The model we will be downloading from Hugging Face is a **5-bit quantized version of the Llama 3 8B chat model**, made available by NousResearch. The model is made available in the **GGUF format** - a new format introduced by the Llama CPP team and a replacement for the earlier GGML format, with advantages such as better tokenization and support for special tokens. Llama is short for **L**arge **LA**nguage Model **M**eta **A**I. https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct-GGUF

*   RAG combines a powerful transformer-based language model with a retrieval system, allowing the model to pull in relevant external information during the generation process. This combination is particularly potent for domains like law where precedent and specific details are crucial. https://www.llamaindex.ai/blog/a-cheat-sheet-and-some-recipes-for-building-advanced-rag-803a9d94c41b

*   FAISS (Facebook AI Similarity Search) empowers us with its state-of-the-art similarity search capabilities, allowing us to effortlessly find documents that closely match a given query. https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/

*   LangChain equips us with advanced text generation techniques, enabling our query engine to generate meaningful and context-aware responses. https://python.langchain.com/v0.1/docs/get_started/introduction

### Dataset

*   The legal dataset provided includes the consolidated acts and regulations of Canada in both English and French as a collection of XML documents which are regularly updated in the linked repository. This dataset will serve as the source for the retrieval component of the RAG, enabling the LLaMA3 model to access and utilize specific legal information when generating responses. https://github.com/justicecanada/laws-lois-xml

# Setup

If you are running this notebook in Google Colab, begin by verifying you  have set your runtime correctly: Runtime > Change runtime type > T4 GPU (or higher)

# Data Download & Preprocessing

We will be utilizing FAISS as our vector store. To begin we must install the required prerequisite libraries and process and embed our XML document data into a FAISS index.

In [1]:
# I borrowed this code from another project in an attempt to understand how to parse all of our XML files into the vector store. We will need to swap the JSON loader for an XML loader at the very least.

import functools
import pathlib
from typing import Any

import langchain.text_splitter
import langchain_community.document_loaders
import langchain_community.embeddings
import langchain_community.vectorstores.faiss
import langchain_core.documents
import sentence_transformers
import torch

# A custom configuration of the SentenceTransformer class is defined using functools.partial, specifying the transformer should run on the CPU and trust remote code.
SentenceTransformer = functools.partial(sentence_transformers.SentenceTransformer, trust_remote_code=True, device="cpu")

# This function is intended to update metadata for a record by extracting and modifying metadata information based on the record's contents:
# - It adjusts the topic by extracting the parent directory name from the source if available.
# - Updates the source and title from the record data.

def metadata_func(record: dict[str, Any], metadata: dict[str, Any]) -> dict[str, Any]:
    """Extract metadata from a record.

    Args:
        record: The record.
        metadata: The default metadata generated by the JSONLoader.

    Returns:
        The updated metadata.
    """
    if "source" in metadata:
        metadata["topic"] = pathlib.Path(metadata["source"]).parent.name
    metadata["source"] = record.get("url")
    metadata["title"] = record.get("title")

    return metadata

# This function deduplicates document chunks:
# - It takes a list of document chunks and filters out duplicates based on the content of the chunks (page_content), ensuring that only unique chunks are retained.

def dedup_chunks(
    chunks: list[langchain_core.documents.Document],
) -> list[langchain_core.documents.Document]:
    """Deduplicate chunks based on their page content.

    Args:
        chunks: A list of chunks.

    Returns:
        A list of deduplicated chunks.
    """
    deduped_chunks = []

    chunk_set = set()
    for chunk in chunks:
        if chunk.page_content not in chunk_set:
            chunk_set.add(chunk.page_content)
            deduped_chunks.append(chunk)
    return deduped_chunks

# The core function of the script, which orchestrates the creation of a FAISS database:
# - Loading Documents: Utilizes DirectoryLoader to load documents from a specified directory. The documents are expected to be in JSON format.
# - Splitting Documents: Documents are split into smaller chunks using a SentenceTransformersTokenTextSplitter, which leverages a specified embedding model.
# - Deduplication: The chunks are deduplicated using the dedup_chunks function.
# - Embedding Documents: Document chunks are embedded using the specified transformer model. Optionally, the model can be set to use half precision.
# - Creating FAISS Index: A FAISS index is created from the deduplicated and embedded chunks, which is then saved locally.

def create_db(
    data_path: str = "/data_fast/laws-lois-xml/documents",
    embedding_model: str = "NousResearch/Meta-Llama-3-8B-Instruct-GGUF",
    save_path: str = "/data_fast/laws-lois-xml/mxbai-embed-large-v1/faiss",
    half_precision: bool = False,
) -> langchain_community.vectorstores.faiss.FAISS:
    """Create a faiss db from a directory of JSON files.

    Args:
        data_path: Path to the directory of JSON files.
        embedding_model: The HuggingFace model name to use for embeddings.
        save_path: Path to save the db.
        half_precision: Whether to use half precision for the embedding model.

    Returns:
        A faiss db.
    """
    loader = langchain_community.document_loaders.DirectoryLoader(
        data_path,
        glob="**/*.json",
        loader_cls=langchain_community.document_loaders.JSONLoader,  # pyright: ignore[reportArgumentType]
        loader_kwargs={
            "jq_schema": ".",
            "metadata_func": metadata_func,
            "content_key": "article",
        },
        use_multithreading=True,
        recursive=True,
    )
    docs = loader.load()
    with mock.patch.object(sentence_transformers, "SentenceTransformer", new=SentenceTransformer):
        splitter = langchain.text_splitter.SentenceTransformersTokenTextSplitter(model_name=embedding_model)
    chunks = splitter.split_documents(docs)
    deduped_chunks = dedup_chunks(chunks)

    embedder = langchain_community.embeddings.HuggingFaceEmbeddings(
        model_name=embedding_model,
        show_progress=True,
        model_kwargs={"trust_remote_code": True},
    )
    assert isinstance(embedder.client, torch.nn.Module)
    if half_precision:
        embedder.client.half()
    db = langchain_community.vectorstores.faiss.FAISS.from_documents(deduped_chunks, embedder)
    db.save_local(save_path)
    return db



ModuleNotFoundError: No module named 'langchain'

# Hugging Face Hub and LangChain Installation

In [4]:
# For downloading the models from HF Hub
!pip install huggingface_hub

In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


# Downloading the Llama 3 8B GGUF model from NousResearch on Hugging Face

The model we will be downloading from Hugging Face is a **5-bit quantized version of the Llama 3 8B chat model**, made available by NousResearch. The model is made available in the **GGUF format** - a new format introduced by the Llama CPP team and a replacement for the earlier GGML format, with advantages such as better tokenization and support for special tokens.

In [2]:
from huggingface_hub import hf_hub_download

In [3]:
model_name_or_path = "NousResearch/Meta-Llama-3-8B-Instruct-GGUF"
model_basename = "Meta-Llama-3-8B-Instruct-Q5_K_M.gguf" # the model is in gguf format

In [6]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Meta-Llama-3-8B-Instruct-Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

# Setting up the Llama 3 Model's Prompt Parameters

We will utilize the Conversation Buffer Window Memory style from LangChain's Memory module. This style of memory only keeps a record of the last K chat interactions (in the style of a sliding window, in this example K = 5), and is hence a controllable and RAM-efficient way of storing memory for our LangChain agent.

In [ ]:
# not really sure what to do here since this model isn't on the list of langchain integrations https://python.langchain.com/v0.1/docs/integrations/llms/

---

#Takeaways
While the Llama 3 LangChain agent is definitely capable of providing answers and also using the external RAG vector store to compute the right answer to the prompt, the LangChain ReAct Prompt Template is very specific and it seems the LLM's chain does not stop even after it arrives at the correct answer.

This is an issue with working with open-source LLMs in combination with LangChain on the free tier of Google Colab - due to the Colab GPU's 13 GB memory limit on the free tier, we are restricted to working with the 8B model of Llama 3, which is not as good at following instructions as OpenAI's GPT models.